# diff3words

a navigation algorithm for what3words based on Levenshtein edit distance

Final Project for Locative Media (NWMEDIA205) at UC Berkeley Fall 2021

created by Chirasree Mandal, December 13, 2021

## Algorithm Sketch

1. Calculate the string edit distance between 2 what3words addresses, `src` and `dest`. Keep track of the insertions/deletions/substitions made at each step and store in an ordered array of strings `steps`.
2. Initalize an empty array `data`
3. For each string in `steps`, determine if it is a valid what3words address. If it is, append it to in `data`
4. Translate each address in `data` to a `(lat, long)` pair using the what3words API
5. Plot each step in `data` on a map. 
5. Plot the route from `src` to `dest` using addresses in `data` on a map

In [1]:
import pandas as pd
import editdistance
import what3words 
import folium
import json
from geopy import distance

In [2]:
# Read in what3words API key
f = open('keys.json')
api_key = json.load(f)["api_key"]
f.close()

In [3]:
# Set up color constants
w3w_red = "#CF3732"
dark_blue = "#0072BB"
yellow = "#e4cc37"
lt_blue = "#1e91d6"
green = "#8fc93a"

To convert 3 word addresses to coordinates, we'll be using the [what3words python API](https://developer.what3words.com/tutorial/python)

In [4]:
# Set up what3words geocoder for API calls
geocoder = what3words.Geocoder(api_key)

In [5]:
# Set up algorithm
def edit_distance_map(steps):
    """Generates a folium Map object that shows the path between src, dest using strings from steps that are valid
        what3words addresses.
    Inputs: 
        steps (array of strings): 
            - ordered array of strings that are the institial string transformations from the edit distance calculation from src to dest. 
            - Assumes steps[0] == src, steps[-1] == dest
            - Assumes steps[0] and steps[-1] are valid what3words addresses, but the interstitial steps may or may not be.
    Returns:
        m (folium.Map)
        df (pandas.DataFrame) contains each step
        """
    # We need at least a source and destination
    if len(steps) < 2:
        print("Please input at least 2 steps.")
        return None
    data = []
    # data will contain sub-arrays of [code (str), lat (float), long (float), nearestPlace (str), w3w url (str)]
    for i in steps:
        res = geocoder.convert_to_coordinates(i)
        # if API result non-null
        if res.get("country"): 
            lat = res.get("coordinates").get("lat")
            long = res.get("coordinates").get("lng")
            np = res.get("nearestPlace")
            url = res.get("map")
            data_point = [i, lat, long, np, url]
            data.append(data_point)
    
    # return early if we don't have a valid start and end point
    if len(data) < 2:
        print("Please specify a valid src and dest what3words code")
        return None    
    src = data[0][0]
    dest = data[-1][0]
            
    total_steps = len(data) - 1
    src_lat_long = (data[0][1], data[0][2])
    dest_lat_long = (data[total_steps][1], data[total_steps][2])
    
    # print information on distances
    
    print("Source: ///{0}, lat: {1}, long: {2}, near {3}".format(src, src_lat_long[0], src_lat_long[1], data[0][3]))
    print("Dest: ///{0}, lat: {1}, long: {2}, near {3}".format(dest, dest_lat_long[0], dest_lat_long[1], data[total_steps][3]))
    print("Geodesic Distance: {:,.2f} miles".format(distance.distance(src_lat_long, dest_lat_long).miles))
    print("Levenshtein Edit Distance: {}".format(editdistance.eval(src, dest)))

    # create a dataframe to organize step data
    df = pd.DataFrame(data, columns = ["words", "lat", "long", "nearestPlace", "url"])
    # get the average lat/long to center the map
    df_avg = df.mean().values
    save_df = df

    #create a folium map
    m = folium.Map(location=df_avg, zoom_start=2, tiles="Stamen Toner")
    
    # draw a Polyline path between src and dest using all of the points
    # https://deparkes.co.uk/2016/06/03/plot-lines-in-folium/
    
    points = df[["lat", "long"]].to_numpy()
    folium.PolyLine(points, color=w3w_red, opacity=0.7).add_to(m)
    
    # add a marker for each step, add its what3words link using popups
    for index, row in df.iterrows():
        name = row["words"]
        url = row["url"]
        if name == src:
            colors = dark_blue
        elif name == dest:
            colors = yellow
        else:
            colors = w3w_red
        folium.RegularPolygonMarker(location=[row["lat"], row["long"]], number_of_sides=4, rotation=45, radius=5, 
                                popup="Step {2}/{3} <a href={0}>///{1}</a>".format(url, name, index, total_steps), 
                                tooltip=None, color=colors, fill_color=colors, opacity=1.0).add_to(m)
    return m, df

## First Example

Source: `///clip.apples.leaps` 

Dest: `///dips.glue.fret`


In [6]:
ex1_source = "clip.apples.leaps"
ex1_dest = "dips.glue.fret"

In [7]:
# Steps
# https://phiresky.github.io/levenshtein-demo/
steps = [ex1_source, 
"dlip.apples.leaps",
"dip.apples.leaps",
"dips.apples.leaps",
"dips.gpples.leaps",
"dips.glples.leaps",
"dips.glules.leaps",
"dips.glues.leaps",
"dips.glue.leaps",
"dips.glue.feaps",
"dips.glue.freaps",
"dips.glue.fretps",
"dips.glue.frets",
ex1_dest]

In [8]:
ex1_map, ex1_df = edit_distance_map(steps)
ex1_map

Source: ///clip.apples.leaps, lat: 38.975623, long: -75.811633, near Greensboro, Maryland
Dest: ///dips.glue.fret, lat: -25.585638, long: 115.0748, near Carnarvon, Western Australia
Geodesic Distance: 11,315.16 miles
Levenshtein Edit Distance: 13


In [9]:
ex1_df

,words,lat,long,nearestPlace,url
0,clip.apples.leaps,38.975623,-75.811633,"Greensboro, Maryland",https://w3w.co/clip.apples.leaps
1,dips.apples.leaps,47.724302,-122.081188,"Cottage Lake, Washington",https://w3w.co/dips.apples.leaps
2,dips.glues.leaps,42.469181,-97.117928,"Hartington, Nebraska",https://w3w.co/dips.glues.leaps
3,dips.glue.leaps,53.398434,-2.817250,"Huyton, Knowsley",https://w3w.co/dips.glue.leaps
4,dips.glue.frets,19.469909,55.063444,"Haymā’, Al Wusta",https://w3w.co/dips.glue.frets
5,dips.glue.fret,-25.585638,115.074800,"Carnarvon, Western Australia",https://w3w.co/dips.glue.fret


In [10]:
#ex1_map.save("clip-apples-leap-stamen-toner-map.html")

## Example 2 -- Nearby Squares

Let's try two nearby locations on the Berkeley campus: Mccone Hall to Moffitt Library

 `///desks.fish.sleepy` to `///gets.packet.text`

In [11]:
mccone_hall = "desks.fish.sleepy"
moffitt_library = "gets.packet.text"


steps2 = [mccone_hall, 
"gesks.fish.sleepy",
"getks.fish.sleepy",
"gets.fish.sleepy",
"gets.pish.sleepy",
"gets.pash.sleepy",
"gets.pach.sleepy",
"gets.pack.sleepy",
"gets.packesleepy",
"gets.packetleepy",
"gets.packet.eepy",
"gets.packet.tepy",
"gets.packet.texy",
moffitt_library]

In [12]:
m2, df2 = edit_distance_map(steps2)
m2

Source: ///desks.fish.sleepy, lat: 37.874178, long: -122.259682, near Berkeley, California
Dest: ///gets.packet.text, lat: 37.872372, long: -122.260878, near Berkeley, California
Geodesic Distance: 0.14 miles
Levenshtein Edit Distance: 13


In [13]:
#m2.save("mccone-moffitt.html")